<table width="100%">
    <tr style="border-bottom:solid 2pt #009EE3">
        <td class="header_buttons">
            <a href="tachogram.zip" download><img src="../../images/icons/download.png" alt="biosignalsnotebooks | download button"></a>
        </td>
        <td class="header_buttons">
            <a href="https://mybinder.org/v2/gh/biosignalsplux/biosignalsnotebooks/mybinder_complete?filepath=biosignalsnotebooks_environment%2Fcategories%2FPre-Process%2Ftachogram.dwipynb" target="_blank"><img src="../../images/icons/program.png" alt="biosignalsnotebooks | binder server" title="Be creative and test your solutions !"></a>
        </td>
        <td></td>
        <td class="header_icons">
            <a href="../MainFiles/biosignalsnotebooks.ipynb"><img src="../../images/icons/home.png" alt="biosignalsnotebooks | home button"></a>
        </td>
        <td class="header_icons">
            <a href="../MainFiles/contacts.ipynb"><img src="../../images/icons/contacts.png" alt="biosignalsnotebooks | contacts button"></a>
        </td>
        <td class="header_icons">
            <a href="https://github.com/biosignalsplux/biosignalsnotebooks" target="_blank"><img src="../../images/icons/github.png" alt="biosignalsnotebooks | github button"></a>
        </td>
        <td class="header_logo">
            <img src="../../images/ost_logo.png" alt="biosignalsnotebooks | project logo">
        </td>
    </tr>
</table>

<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_4"><div id="image_img" class="header_image_4"></div></td>
        <td class="header_text"> Generation of Tachogram from ECG </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">pre-process&#9729;ecg&#9729;tachogram</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

Like described on another <strong><span class="color5">Jupyter Notebook</span></strong> entitled <a href="../Detect/r_peaks.ipynb">"Event Detection - R Peaks (ECG)" <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>, periodicity is one of the distinctive characteristics of electrocardiographic signals.

For studying periodicity it is necessary to detect a specific event in all the cardiac cycles. The <strong><span class="color7">QRS complex</span></strong> is the most prominent structure of ECG signal, being the event commonly chosen. 

After detecting each R peak the time interval between them can be determined. With each time interval a time series may be generated, named <strong><span class="color2">Tachogram</span></strong>. 

Essentially, the tachogram describes how the cardiac period changes along time and is the inverse of heart rate time series.

Here, it will be presented a simple way to generate a tachogram from the acquired data.

<hr>

<p class="steps">1 - Importation of the needed packages</p>

In [1]:
# biosignalsnotebooks own package for loading and plotting the acquired data
import biosignalsnotebooks as bsnb

# Scientific packages
import numpy

<p class="steps">2 - Load of acquired ECG data</p>

In [2]:
# Load of data
data, header = bsnb.load("../../signal_samples/ecg_4000_Hz.h5", get_header=True);

<p class="steps">3 - Identification of mac address of the device and the channel used during acquisition</p>
Both data and header are store in <a href="https://www.w3schools.com/python/python_dictionaries.asp">Python dictionaries</a>, so, we must retrieve the data contained in those dictionaries.

In [3]:
# Data is in dictionary format and channel is one of the keys. This line gets the first key of the dictionary.
channel = list(data.keys())[0]

# The mac address of the acquiring device corresponds to the device name of the dictionary returned in the header, that was previously stored in a variable.
mac_address = str(header["device name"])

In [4]:
print ("Mac Address: " + mac_address + " Channel: " + str(channel))

Mac Address: b'00:07:80:D8:A7:F9' Channel: CH1


<p class="steps">4 - Storage of sampling frequency and acquired data inside variables</p>

In [5]:
# Sampling frequency of acquired data
fs = header["sampling rate"]

# Signal Samples
signal = data[channel]

# Generate the time axis of the signal given its sampling frequency
time = bsnb.generate_time(signal, fs)

<p class="steps">5 - Detection of the time instants where R peaks are located</p>
The detection of the R peaks is executed using the <a href="https://en.wikipedia.org/wiki/Pan%E2%80%93Tompkins_algorithm">Pan-Tompkins algorithm</a>, which is computationally inexpensive while offering high accuracy for the detection. This algorithm is implemented in the <strong><span class="color2">biosignalsnotebooks</span></strong> Python package, facilitating its application.

In [6]:
# R peak detection.
time_r_peaks, amp_r_peaks = bsnb.detect_r_peaks(signal, fs, time_units=True)

<p class="steps">6 - Determination of the time between consecutive R peaks (cardiac cycle)</p>
The <strong><span class="color4">tachogram</span></strong> corresponds to the derivative of the signal, i.e., to the interval duration of peaks over time, in order to get the <strong>variation</strong> of the interval duration of peaks over time (or the variation of the heartbeats period over time). The function <strong>diff</strong>, provided in numpy, computes the difference between successive data in an array. Thus, the tachogram can be computed by applying the diff function to the time of the detected peaks, which results in the interval between consecutive R peaks.

In [7]:
tachogram = numpy.diff(time_r_peaks)

# The tachogram time can be obtained by shifting each point of heartbeat duration to the center of the two corresponding peaks.
tachogram_time = (time_r_peaks[1:] + time_r_peaks[:-1]) / 2

<p class="steps">7 - Representation of tachogram samples and highlighting of the respective cardiac cycle in the bottom plot</p>
The next figures illustrate the tachogram and the corresponding ECG signal that originated it. The areas of the plots with different colors correspond to different heartbeats and indicate the interval considered to calculate the duration of each point in the tachogram.

In [8]:
list_figures = bsnb.plot_ecg_tachogram(time, signal, tachogram_time, tachogram, time_r_peaks)

*This procedure can be automatically done by **tachogram** function in **detect** module of **<span class="color2">biosignalsnotebooks</span>** package*.

In [9]:
tachogram_data, tachogram_time = bsnb.tachogram(signal, fs, signal=True, out_seconds=True)

As demonstrated in this <strong><span class="color5">Jupyter Notebook</span></strong>, the simplicity of generating a tachogram as a visualisation support to analyse cardiac events, namely, heart period over time, allows to access information that is not directly observable from the ECG signal, but that may still be relevant.

Using the <strong><span class="color2">biosignalsnotebooks</span></strong> Python package, the procedure can even be reduced to one line of code!

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> ! 

<hr>
<table width="100%">
    <tr>
        <td class="footer_logo">
            <img src="../../images/ost_logo.png" alt="biosignalsnotebooks | project logo [footer]">
        </td>
        <td width="40%" style="text-align:left">
            <a href="../MainFiles/aux_files/biosignalsnotebooks_presentation.pdf" target="_blank">&#9740; Project Presentation</a>
            <br>
            <a href="https://github.com/biosignalsplux/biosignalsnotebooks" target="_blank">&#9740; GitHub Repository</a>
            <br>
            <a href="https://pypi.org/project/biosignalsnotebooks/" target="_blank">&#9740; How to install biosignalsnotebooks Python package ?</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/signal_samples.ipynb">&#9740; Signal Library</a>
        </td>
        <td width="40%" style="text-align:left">
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/biosignalsnotebooks.ipynb">&#9740; Notebook Categories</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/by_diff.ipynb">&#9740; Notebooks by Difficulty</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/by_signal_type.ipynb">&#9740; Notebooks by Signal Type</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/by_tag.ipynb">&#9740; Notebooks by Tag</a>
        </td>
    </tr>
</table>

In [10]:
from biosignalsnotebooks.__notebook_support__ import css_style_apply
css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................


In [11]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>